# 2A.dl - Deep learning avec Python

Quelques librairies de deep learning sous Python.

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

## theano

[documentation](http://deeplearning.net/software/theano/index.html) [tutorial MNIST](http://deeplearning.net/tutorial/logreg.html) [tutorial Logisitic Regression](http://deeplearning.net/tutorial/logreg.html)

Le module [theano](http://deeplearning.net/software/theano/) propose une API pour écrire des algorithmes numériques utilisant le [GPU](https://en.wikipedia.org/wiki/Graphics_processing_unit). Il n'est pas spécialisé dans le deep learning même s'il propose de nombreuses fonctionnalités pour cela. Le module implémente ses propres container. Il faut voir [theano](http://deeplearning.net/software/theano/) comme une sorte de *numpy GPU*. Le module est écrit de telle sorte qu'il cherche à optimiser la vitesse de calcul en transformant le code Python en C++ qui est ensuite compilé. Pour l'installation et notamment un compilateur C++, il faut le début de la partie relative au [deep learning](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3/td_2a_mlplus.html#deep-learning).

Il faut chercher la fonction ``sgd_optimization_mnist`` dans l'aide de ce site pour trouver un exemple utilisant *theano* (le tutorial du cours). Il est recommandé d'utiliser les autres librairies telles que [keras](https://keras.io/)...

**L'équipe qui le maintient a décidé son support. Il devrait progressivement cesser de fonctionner en 2018.**

## keras

[documentation](https://keras.io/)

La première étape consiste à définir votre [backend](https://keras.io/backend/), c'est-à-dire la librairie de deep learning que vous souhaitez utiliser.

In [3]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [4]:
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

On s'inspire de l'exemple [mnist_cnn](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). Le [glossaire](http://www.wildml.com/deep-learning-glossary/) suivant vous aidera à comprendre le code.

In [5]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

def keras_mnist_data():
    # the data, shuffled and split between train and test sets
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    img_rows, img_cols = 28, 28    # should be cmputed from the data

    if K.image_dim_ordering() == 'th':
        X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
        X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    else:
        X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
        X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255

    # convert class vectors to binary class matrices
    nb_classes = len(set(y_train))
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)

    return (X_train, Y_train), (X_test, Y_test)

(X_train, Y_train), (X_test, Y_test) = keras_mnist_data()

In [6]:
def keras_build_mnist_model(nb_classes):
    model = Sequential()

    nb_filters = 32
    pool_size = (2, 2)
    kernel_size = (3, 3)
    img_rows, img_cols = 28, 28    # should be cmputed from the data

    print("[keras_build_mnist_model] K.image_dim_ordering()={0}".format(
        K.image_dim_ordering()))
    if K.image_dim_ordering() == 'th':
        input_shape = (1, img_rows, img_cols)
    else:
        input_shape = (img_rows, img_cols, 1)

    model.add(Conv2D(nb_filters, kernel_size, padding='valid', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(nb_filters, kernel_size))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    return model

model = keras_build_mnist_model(Y_train.shape[1])
model

[keras_build_mnist_model] K.image_dim_ordering()=tf


L'étape suivante peut être très longue. Il est recommandé que le code soit compilé. Il faut changer ``nb_epoch`` à 12 pour obtenir une bonne performance.

In [7]:
def keras_fit(model, X_train, Y_train, X_test, Y_test, batch_size=128,
              nb_classes=None, epochs=12):
    if nb_classes is None:
        nb_classes = Y_train.shape[1]
        print("[keras_fit] nb_classes=%d" % nb_classes)
    model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
              verbose=1, validation_data=(X_test, Y_test))
    
keras_fit(model, X_train, Y_train, X_test, Y_test, batch_size=128,
          nb_classes=None, epochs=1)    

[keras_fit] nb_classes=10
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 123s - loss: 0.3694 - acc: 0.8873 - val_loss: 0.0883 - val_acc: 0.9721


In [8]:
def keras_predict(model, X_test, Y_test):
    return model.evaluate(X_test, Y_test, verbose=0)
score = keras_predict(model, X_test, Y_test)
score[:5]

[0.088284374502673749, 0.97209999999999996]

## tensorflow

[tensorflow](https://www.tensorflow.org/)

## CNTK

[CNTK](https://docs.microsoft.com/en-us/cognitive-toolkit/)

## pytorch

[pytorch](https://pytorch.org/)

## Comparatifs

* [DL4J vs. Torch vs. Theano vs. Caffe vs. TensorFlow](https://deeplearning4j.org/compare-dl4j-torch7-pylearn)
* [Evaluation of Deep Learning Toolkits](https://github.com/zer0n/deepframeworks/blob/master/README.md)
* [Benchmarking CNTK on Keras: is it Better at Deep Learning than TensorFlow?](http://minimaxir.com/2017/06/keras-cntk/) (2017)

**END**